# Read SEG-Y with segyio

This is a relatively new library from Statoil. It is very easy to use... in most cases.

In [ ]:
import segyio

In [ ]:
help(segyio)

## Basics

If you don't have the file yet, **[get the large dataset from Agile's S3 bucket](https://s3.amazonaws.com/agilegeo/Penobscot_0-1000ms.sgy.zip)**. It's 140MB.

In [ ]:
with segyio.open('../data/Penobscot_0-1000ms.sgy') as s:
    print("Binary header")
    print(s.bin)
    print()
    print("Text header")
    print(s.text[0])

This garbled text header is a bug. `segyio` currently (Jan 2019) assumes the header is EBCDIC encoded, but in this file it's ASCII encoded. It has been filed [as an issue](https://github.com/equinor/segyio/issues/317).

## Access the data

In [ ]:
with segyio.open('../data/Penobscot_0-1000ms.sgy') as s:
    c = segyio.cube(s)

`c` is just an `ndarray`.

In [ ]:
type(c)

In [ ]:
c.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(c[100].T, cmap='Greys')

## 2D data

https://github.com/agile-geoscience/geocomputing/blob/master/data/HUN00-ALT-01_STK.sgy

This file does not open with the default `strict=True`:

In [ ]:
# This should produce an error.
with segyio.open('../data/HUN00-ALT-01_STK.sgy') as s:
    c = segyio.cube(s)

It's OK if not strict... but then you can't use `cube`

In [ ]:
with segyio.open('../data/HUN00-ALT-01_STK.sgy', strict=False) as s:
    c = segyio.cube(s)

So we'll unpack the traces manually... 

In [ ]:
import numpy as np

with segyio.open('../data/HUN00-ALT-01_STK.sgy', strict=False) as s:
    data = np.stack(t.astype(np.float) for t in s.trace)

In [ ]:
plt.figure(figsize=(15,5))
plt.imshow(data.T)

With a bit more work we can also read the file header and the trace headers.

In [ ]:
import numpy as np

def chunks(s, n):
    """Produce `n`-character chunks from string `s`."""
    for start in range(0, len(s), n):
        yield s[start:start + n]

with segyio.open('../data/HUN00-ALT-01_STK.sgy', strict=False) as s:
    
    # Read the data.
    data = np.stack(t.astype(np.float) for t in s.trace)
    
    # Get the (x, y) locations.
    x = [t[segyio.TraceField.GroupX] for t in s.header]
    y = [t[segyio.TraceField.GroupY] for t in s.header]
    
    # Get the trace numbers.
    cdp = np.array([t[segyio.TraceField.CDP] for t in s.header])

    # Get the first textual header.
    header = s.text[0].decode('ascii')
    formatted = '\n'.join(chunk for chunk in chunks(header, 80))

    # Get data from the binary header.
    # Get the sample interval in ms (convert from microsec).
    sample_interval = s.bin[segyio.BinField.Interval] / 1000

print(formatted)

Getting a sub-set of traces using CDP (or trace number or similar) is a little fiddly:

In [ ]:
cdp

In [ ]:
selection = np.where((cdp>500) & (cdp<800))[0]
subset = data[selection]

plt.imshow(subset.T, aspect='auto')

## Try another

In [ ]:
with segyio.open('../data/31_81_PR.sgy') as s:
    data = segyio.cube(s)

Nope.

In [ ]:
with segyio.open('../data/31_81_PR.sgy', strict=False) as s:
    data = np.stack(t.astype(np.float) for t in s.trace)

In [ ]:
data.shape

OK, I guess this isn't quite the flow for a 2D file... 

In [ ]:
plt.figure(figsize=(16, 8))
plt.imshow(np.squeeze(data).T, cmap='Greys', aspect=0.2)

## Another, known to be 'weird'

In [ ]:
with segyio.open('../data/31_81_PR.sgy') as s:
    data = segyio.cube(s)

Nope again.

In [ ]:
with segyio.open('../data/marmousi/velocity.segy', strict=False) as s:
    print("weird dt: ", segyio.dt(s))
    data = np.stack(t.astype(np.float) for t in s.trace)

This file is improperly organized (time first, so we don't need to transpose it) and the dt header is wrong.

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(data, cmap='viridis', aspect='auto')
plt.show()